<h1>Chinook</h1>

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases.

Database from [Github](https://github.com/lerocha/chinook-database), modified and provided by DataQuest

In [1]:
!conda install -yc conda-forge ipython-sql

Solving environment: ...working... done

# All requested packages already installed.



In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

<h4>List all tables and views in database</h4>

In [3]:
%%sql
SELECT name,
       type
  FROM sqlite_master
 WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


<h2>Music</h2>
<h4>Top 5 albums</h4>
By number of times a track from that album has been purchased

In [4]:
%%sql
WITH ta AS (
        SELECT t.track_id,
               al.title AS album_title,
               ar.name AS artist_name
          FROM track AS t
    INNER JOIN album AS al ON al.album_id = t.album_id
    INNER JOIN artist AS ar ON ar.artist_id = al.artist_id) 


    SELECT ta.album_title AS album,
           ta.artist_name AS artist,
           COUNT(*) AS tracks_purchased
      FROM invoice_line AS il
INNER JOIN ta ON ta.track_id = il.track_id
  GROUP BY 1, 2
  ORDER BY 3 DESC 
     LIMIT 5;

 * sqlite:///chinook.db
Done.


album,artist,tracks_purchased
Are You Experienced?,Jimi Hendrix,187
Faceless,Godsmack,96
Mezmerize,System Of A Down,93
Get Born,JET,90
The Doors,The Doors,83


<h4>Top 10 genres that sell the most tracks in the USA</h4>

In [5]:
%%sql
WITH usa_tracks_sold AS (
        SELECT il.* 
          FROM invoice_line AS il
    INNER JOIN invoice AS i on il.invoice_id = i.invoice_id
    INNER JOIN customer AS c on i.customer_id = c.customer_id
         WHERE c.country = "USA")


    SELECT g.name genre,
           count(uts.invoice_line_id) tracks_sold,
           ROUND((CAST(COUNT(uts.invoice_line_id) AS Float) 
               / (SELECT COUNT(*) from usa_tracks_sold) * 100), 2)
               AS percentage_sold
      FROM usa_tracks_sold uts
INNER JOIN track t on t.track_id = uts.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
  GROUP BY 1
  ORDER BY 2 DESC
     LIMIT 10;

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Pop,22,2.09
Latin,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


<h2>Employees</h2>
<h4>Information about employee and supervisor</h4>

In [6]:
%%sql
   SELECT e1.first_name || " " || e1.last_name AS employee_name,
          e1.title AS employee_title,
          e2.first_name || " " || e2.last_name AS supervisor_name,
          e2.title AS supervisor_title
     FROM employee AS e1
LEFT JOIN employee AS e2 ON e1.reports_to = e2.employee_id
 ORDER BY 1;

 * sqlite:///chinook.db
Done.


employee_name,employee_title,supervisor_name,supervisor_title
Andrew Adams,General Manager,None,None
Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
Laura Callahan,IT Staff,Michael Mitchell,IT Manager
Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
Michael Mitchell,IT Manager,Andrew Adams,General Manager
Nancy Edwards,Sales Manager,Andrew Adams,General Manager
Robert King,IT Staff,Michael Mitchell,IT Manager
Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager


<h4>Best Sales Support Agents based on total_sales</h4>

In [7]:
%%sql
WITH support_agents AS (
         SELECT i.customer_id,
                c.support_rep_id,
                CAST(SUM(i.total) AS Integer) AS total
           FROM invoice AS i
     INNER JOIN customer AS c ON i.customer_id = c.customer_id
     GROUP BY 1,2)


    SELECT e.first_name || " " || e.last_name employee,
           e.hire_date,
           SUM(sa.total) total_sales
      FROM support_agents AS sa
INNER JOIN employee e ON e.employee_id = sa.support_rep_id
  GROUP BY 1;

 * sqlite:///chinook.db
Done.


employee,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1723
Margaret Park,2017-05-03 00:00:00,1577
Steve Johnson,2017-10-17 00:00:00,1389


<h2>Customers</h2>
<h4>Expenses by customers</h4>

In [8]:
%%sql
    SELECT c.first_name || " " || c.last_name AS customer_name,
           COUNT(i.invoice_id) AS number_of_purchases,
           ROUND(SUM(i.total), 2) AS total_spent,
           CASE
           WHEN sum(i.total) < 40 THEN 'small spender'
           WHEN sum(i.total) > 100 THEN 'big spender'
           ELSE 'regular'
           END AS customer_category
      FROM invoice AS i
INNER JOIN customer AS c ON i.customer_id = c.customer_id
  GROUP BY 1 
  ORDER BY 4;

 * sqlite:///chinook.db
Done.


customer_name,number_of_purchases,total_spent,customer_category
Fernanda Ramos,15,106.92,big spender
František Wichterlová,18,144.54,big spender
Helena Holý,12,128.7,big spender
Hugh O'Reilly,13,114.84,big spender
João Fernandes,13,102.96,big spender
Luís Gonçalves,13,108.9,big spender
Manoj Pareek,13,111.87,big spender
Aaron Mitchell,8,70.29,regular
Alexandre Rocha,10,69.3,regular
Astrid Gruber,9,69.3,regular


<h4>Customers that have purchased more than $90</h4>

In [9]:
%%sql
CREATE VIEW customer_gt_90_dollars AS
        SELECT c.first_name || " " || c.last_name AS customer_name,
               c.phone,
               c.email,
               ROUND(SUM(i.total), 2) AS total_spent
          FROM invoice i
    INNER JOIN customer AS c ON i.customer_id = c.customer_id
      GROUP BY 1
        HAVING SUM(i.total) > 90;

        
SELECT *
  FROM customer_gt_90_dollars;

 * sqlite:///chinook.db
(sqlite3.OperationalError) table customer_gt_90_dollars already exists
[SQL: CREATE VIEW customer_gt_90_dollars AS
        SELECT c.first_name || " " || c.last_name AS customer_name,
               c.phone,
               c.email,
               ROUND(SUM(i.total), 2) AS total_spent
          FROM invoice i
    INNER JOIN customer AS c ON i.customer_id = c.customer_id
      GROUP BY 1
        HAVING SUM(i.total) > 90;]
(Background on this error at: http://sqlalche.me/e/e3q8)


<h4>Total_purchases from customers from India</h4> 

In [10]:
%%sql
WITH
    customers_india AS (
           SELECT * 
             FROM customer
            WHERE country = "India"),
    sales_per_customer AS (
           SELECT customer_id,
                  CAST(SUM(total) as Integer) total
             FROM invoice
         GROUP BY 1)

    
    SELECT ci.first_name || " " || ci.last_name AS customer_name,
           spc.total AS total_purchases
      FROM customers_india AS ci
INNER JOIN sales_per_customer AS spc ON ci.customer_id = spc.customer_id
  ORDER BY 1;

 * sqlite:///chinook.db
Done.


customer_name,total_purchases
Manoj Pareek,111
Puja Srivastava,71


<h4>Customer that spend the most per country</h4>

In [11]:
%%sql
WITH
    customer_country_purchases AS (
             SELECT i.customer_id,
                    c.country,
                    CAST(SUM(i.total) AS Integer) AS total_purchases
               FROM invoice AS i
         INNER JOIN customer AS c ON i.customer_id = c.customer_id
           GROUP BY 1, 2),
    
    country_max_purchase AS (
             SELECT country,
                    MAX(total_purchases) max_purchase
               FROM customer_country_purchases
           GROUP BY 1),
    
    country_best_customer AS (
             SELECT cmp.country,
                    cmp.max_purchase,
                    (SELECT ccp.customer_id
                       FROM customer_country_purchases ccp
                      WHERE ccp.country = cmp.country 
                        AND cmp.max_purchase = ccp.total_purchases) AS customer_id
               FROM country_max_purchase cmp)

    
    SELECT cbc.country AS country,
           c.first_name || " " || c.last_name AS customer_name,
           cbc.max_purchase AS total_purchased
      FROM customer AS c
INNER JOIN country_best_customer AS cbc ON cbc.customer_id = c.customer_id
  ORDER BY 1 ASC;

 * sqlite:///chinook.db
Done.


country,customer_name,total_purchased
Argentina,Diego Gutiérrez,39
Australia,Mark Taylor,81
Austria,Astrid Gruber,69
Belgium,Daan Peeters,60
Brazil,Luís Gonçalves,108
Canada,François Tremblay,99
Chile,Luis Rojas,97
Czech Republic,František Wichterlová,144
Denmark,Kara Nielsen,37
Finland,Terhi Hämäläinen,79
